In [1]:
%load_ext autoreload
%autoreload 2
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
%env XLA_PYTHON_CLIENT_MEM_FRACTION=.80
%env XLA_PYTHON_CLIENT_ALLOCATOR=platform
# %env TF_FORCE_GPU_ALLOW_GROWTH=true
# %env TF_GPU_ALLOCATOR=cuda_malloc_async
%env KERAS_BACKEND=jax
# import tensorflow as tf
from usl_models.flood_ml import constants
from usl_models.flood_ml import model
from usl_models.flood_ml import fake_data
import functools
from typing import Generator, Tuple, Dict, Any
import dataclasses
import itertools
import jax
import jax.numpy as jnp
import tensorflow as tf
tf.config.experimental.set_visible_devices([], "GPU")

env: XLA_PYTHON_CLIENT_PREALLOCATE=false
env: XLA_PYTHON_CLIENT_MEM_FRACTION=.80
env: XLA_PYTHON_CLIENT_ALLOCATOR=platform
env: KERAS_BACKEND=jax


2024-07-04 01:22:54.356270: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 01:22:54.371712: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 01:22:54.371742: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-04 01:22:55.019488: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-07-04 01:22:55.835983: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the gu

In [2]:
import keras
keras.config.disable_traceback_filtering()

In [3]:
config = fake_data.DatasetConfig(width=1024, height=1024, batch_size=1, storm_duration=4)
dataset, storm_duration = fake_data.get_fake_dataset(config)
dataset = dataset.batch(config.batch_size)
dataset

<_BatchDataset element_spec=({'geospatial': TensorSpec(shape=(None, 1, 1024, 1024, 8), dtype=tf.float32, name=None), 'temporal': TensorSpec(shape=(None, 1, 4, 6), dtype=tf.float32, name=None), 'spatiotemporal': TensorSpec(shape=(None, 1, 1024, 1024, 1), dtype=tf.float32, name=None)}, TensorSpec(shape=(None, 1, 4, 1024, 1024), dtype=tf.float32, name=None))>

In [4]:
flood_model = model.FloodModel(
    model_params=model.FloodModelParams(batch_size=config.batch_size), spatial_dims=(config.height, config.width))
flood_model._model.summary()

Model: "flood_conv_lstm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ spatiotemporal_cnn (Sequential) │ (None, None, 256, 256, │         3,424 │
│                                 │ 16)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ geospatial_cnn (Sequential)     │ (None, 256, 256, 64)   │        28,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm (Sequential)          │ (None, 256, 256, 128)  │       986,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_cnn (Sequential)         │ (None, 1024, 1024, 1)  │        16,401 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,035,329 (3.95 MB)

 Trainable params: 1,035,329 (3.95 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
H = 2
W = 3
B = 4

st_input0 = jnp.ones((B, H, W, 1))
st_input = jnp.zeros((B, constants.MAX_TIMESTEPS, H, W, 1))
st_input = st_input.at[:, 0].set(st_input0)

In [6]:
generator = fake_data.fake_example_generator(config)
# X, Y = itertools.tee(generator)
# X, Y = (x[0] for x in X), (y[1] for y in Y)

In [7]:
for x, y in generator:
    pred = flood_model._model.call(x)
    print(pred)
    break


---------------------------------------------------------
Generate sample 0.
st_input0: (1, 1024, 1024, 1)
geo_input: (1, 1024, 1024, 8)
full_temp_input: (1, 4, 6)

----------------------------------------------------------------
N 4
n 1
geo_cnn_output (1, 1, 256, 256, 64)
geo_cnn_output (1, 4, 256, 256, 64)
lstm_input (1, 4, 256, 256, 86)


E0704 01:23:06.422702   22702 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0704 01:23:06.563621   22702 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0704 01:23:06.709156   22702 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


[[[[[-2.074776  ]
    [-0.76736706]
    [ 3.172783  ]
    ...
    [ 0.571809  ]
    [ 0.33215055]
    [ 1.586587  ]]

   [[-1.2886802 ]
    [-0.2322635 ]
    [-0.52876574]
    ...
    [ 0.635303  ]
    [ 1.2648233 ]
    [ 0.89189464]]

   [[-0.47766408]
    [ 1.0721385 ]
    [-0.15676394]
    ...
    [-0.03262298]
    [-0.7714375 ]
    [ 0.85593235]]

   ...

   [[ 1.2903061 ]
    [-0.6115401 ]
    [ 0.5610416 ]
    ...
    [-1.5337036 ]
    [ 2.224471  ]
    [-1.0702143 ]]

   [[-0.664462  ]
    [ 0.34221122]
    [-0.10822076]
    ...
    [-0.10092368]
    [ 0.7209546 ]
    [-0.49332994]]

   [[ 0.6955754 ]
    [-0.14716521]
    [ 0.3615766 ]
    ...
    [-1.475763  ]
    [-2.6703808 ]
    [ 0.07674517]]]


  [[[ 0.01782552]
    [ 0.        ]
    [ 0.        ]
    ...
    [ 0.        ]
    [ 0.        ]
    [ 0.02724046]]

   [[ 0.        ]
    [ 0.05957777]
    [ 0.05042163]
    ...
    [ 0.02127733]
    [ 0.04017482]
    [ 0.02131203]]

   [[ 0.04427752]
    [ 0.        ]
    [ 0.  

In [8]:
flood_model.train(fake_data.fake_example_generator(config), config=config)


---------------------------------------------------------
Generate sample 0.

---------------------------------------------------------
Generate sample 1.
st_input0: (1, 1024, 1024, 1)
geo_input: (1, 1024, 1024, 8)
full_temp_input: (1, 4, 6)

----------------------------------------------------------------
N 4
n 1
geo_cnn_output (1, 1, 256, 256, 64)
geo_cnn_output (1, 4, 256, 256, 64)
lstm_input (1, 4, 256, 256, 86)

----------------------------------------------------------------
N 4
n 1
geo_cnn_output (1, 1, 256, 256, 64)
geo_cnn_output (1, 4, 256, 256, 64)
lstm_input (1, 4, 256, 256, 86)

----------------------------------------------------------------
N 4
n 1
geo_cnn_output (1, 1, 256, 256, 64)
geo_cnn_output (1, 4, 256, 256, 64)
lstm_input (1, 4, 256, 256, 86)

----------------------------------------------------------------
N 4
n 1
geo_cnn_output (1, 1, 256, 256, 64)
geo_cnn_output (1, 4, 256, 256, 64)
lstm_input (1, 4, 256, 256, 86)
Epoch 1/10

---------------------------------

2024-07-04 01:23:18.599253: E external/xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng55{k2=1,k13=0,k14=3,k18=1,k22=0,k23=0} for conv (f32[8,128,4,4]{3,2,1,0}, u8[0]{0}) custom-call(f32[8,1,1024,1024]{3,2,1,0}, f32[128,1,1021,1021]{3,2,1,0}), window={size=1021x1021 rhs_reversal=1x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false} is taking a while...
2024-07-04 01:23:28.225649: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 10.6266163s
Trying algorithm eng55{k2=1,k13=0,k14=3,k18=1,k22=0,k23=0} for conv (f32[8,128,4,4]{3,2,1,0}, u8[0]{0}) custom-call(f32[8,1,1024,1024]{3,2,1,0}, f32[128,1,1021,1021]{3,2,1,0}), window={size=1021x1021 rhs_reversal=1x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__

      1/Unknown 124s 124s/step - loss: 1.2527 - mean_absolute_error: 0.8815 - root_mean_squared_error: 1.1192
---------------------------------------------------------
Generate sample 4.
st_input0: (1, 1024, 1024, 1)
geo_input: (1, 1024, 1024, 8)
full_temp_input: (1, 4, 6)

----------------------------------------------------------------
N 4
n 1
geo_cnn_output (1, 1, 256, 256, 64)
geo_cnn_output (1, 4, 256, 256, 64)
lstm_input (1, 4, 256, 256, 86)

----------------------------------------------------------------
N 4
n 1
geo_cnn_output (1, 1, 256, 256, 64)
geo_cnn_output (1, 4, 256, 256, 64)
lstm_input (1, 4, 256, 256, 86)

----------------------------------------------------------------
N 4
n 1
geo_cnn_output (1, 1, 256, 256, 64)
geo_cnn_output (1, 4, 256, 256, 64)
lstm_input (1, 4, 256, 256, 86)

----------------------------------------------------------------
N 4
n 1
geo_cnn_output (1, 1, 256, 256, 64)
geo_cnn_output (1, 4, 256, 256, 64)
lstm_input (1, 4, 256, 256, 86)
      2/Unknow

{'loss': [1.2507960796356201, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'mean_absolute_error': [0.880704939365387,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'root_mean_squared_error': [1.1183899641036987,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0]}

In [9]:
flood_model._model.summary()

Model: "flood_conv_lstm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ spatiotemporal_cnn (Sequential) │ (None, None, 256, 256, │         3,424 │
│                                 │ 16)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ geospatial_cnn (Sequential)     │ (None, 256, 256, 64)   │        28,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm (Sequential)          │ (None, 256, 256, 128)  │       986,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_cnn (Sequential)         │ (None, 1024, 1024, 1)  │        16,401 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,105,989 (11.85 MB)

 Trainable params: 1,035,329 (3.95 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,070,660 (7.90 MB)